## Imports and download

In [2]:
! pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.2 MB/s eta 0:00:00


In [3]:
# from huggingface_hub import notebook_login

# notebook_login()

In [4]:
import transformers

print(transformers.__version__)

4.30.2


In [5]:
import torch
from torch.optim import AdamW
from tqdm import tqdm

In [6]:
from transformers.utils import send_example_telemetry

send_example_telemetry("language_modeling_notebook", framework="pytorch")

In [7]:
model_checkpoint = "distilgpt2"
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [8]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

## Hook into model and optimize sentence

In [ ]:
def enembed(embeds_input):
  """
  Given an embedding vector, decode each token by using the transpose of the embedding matrix
  and grabbing the vocab token with the highest probability on each token.

  Also do this with the unembedding matrix as well.
  """
  with torch.no_grad():
      # Get the pre-trained embeddings
      pretrained_embeddings = model.transformer.wte.weight

      # Calculate dot product between input embeddings and pre-trained embeddings
      dot_product = torch.matmul(embeds_input, pretrained_embeddings.t())

      # Get the index of the highest value along dimension 2 (tokens)
      _, tokens = torch.max(dot_product, dim=2)

  # Decode tokens into text using the tokenizer
  text = tokenizer.batch_decode(tokens.tolist(), skip_special_tokens=True)

  return text


In [9]:
layer = model.transformer.h[1].mlp

In [10]:
activation_saved = [0.0]
def hook(model, input, output):
  # The output is a tensor. You can index it to get the activation of a specific neuron.
  # Here we're getting the activation of the 0th neuron.
  activation = output[0, 0, 0]  # Gets the the 0th neuron of the 0th example in the batch
  activation_saved[0] = activation


In [14]:
# Register the hook
try:
  handle.remove() # deletes the handle when we are done with it.
except:
  pass
handle = layer.register_forward_hook(hook)

In [27]:
inputs = tokenizer("Hello, world!", return_tensors="pt")

# Get embeddings
with torch.no_grad():
    embeddings = model.transformer.wte(inputs["input_ids"])

# Make embeddings require gradient
embeddings.requires_grad_(True)

# Create an optimizer for the embeddings
optimizer = AdamW([embeddings], lr=1e-3)  # You may need to adjust the learning rate
pre_embeddings = embeddings.detach().clone()
print(embeddings)
print(enembed(pre_embeddings))

tensor([[[-0.0904, -0.1538,  0.0315,  ...,  0.0774, -0.0212, -0.0622],
         [ 0.0086, -0.0009,  0.0056,  ...,  0.0484, -0.0737, -0.0636],
         [-0.1725,  0.1922, -0.0372,  ..., -0.3523,  0.1989,  0.0269],
         [-0.1445, -0.0455,  0.0042,  ..., -0.1523,  0.0184,  0.0991]]],
       requires_grad=True)
['Hello, world!']


In [36]:
dist = 0.0
losses = []
for i in tqdm(range(1000)):
  outputs = model(inputs_embeds=embeddings, attention_mask=inputs.attention_mask)
  loss = activation_saved[0]
  loss.backward()
  optimizer.step()
  dist = torch.sum(embeddings - pre_embeddings).item()
  losses.append(loss)
  if i % 25 == 0:
    tqdm.write(f"\n{dist} and then {loss}")
    tqdm.write(enembed(embeddings)[0])
  optimizer.zero_grad()


  0%|          | 2/1000 [00:00<02:29,  6.69it/s]


2.7882938385009766 and then -19.504741668701172
Hello, world!


  3%|▎         | 26/1000 [00:03<03:53,  4.18it/s]


3.07511568069458 and then -22.590717315673828
Hello, world!


  5%|▌         | 51/1000 [00:10<03:33,  4.45it/s]

...

 95%|█████████▌| 951/1000 [02:05<00:07,  6.24it/s]


5.759215354919434 and then -71.21534729003906
resents, world!


 98%|█████████▊| 977/1000 [02:09<00:03,  7.32it/s]


5.9168291091918945 and then -71.8554916381836
resents, world!


100%|██████████| 1000/1000 [02:12<00:00,  7.56it/s]


In [29]:
embeddings

tensor([[[ 0.0199, -0.0337,  0.0911,  ...,  0.1578, -0.1568, -0.1923],
         [ 0.0086, -0.0009,  0.0056,  ...,  0.0483, -0.0737, -0.0636],
         [-0.1723,  0.1921, -0.0371,  ..., -0.3519,  0.1987,  0.0269],
         [-0.1443, -0.0455,  0.0042,  ..., -0.1521,  0.0184,  0.0990]]],
       requires_grad=True)

In [18]:
embeddings.shape

torch.Size([1, 4, 768])

In [19]:
embed_matrix = model.transformer.wte.weight

In [21]:
embed_matrix.shape

torch.Size([50257, 768])

tensor([[[ 1.1032e-01,  1.2014e-01,  5.9592e-02,  ...,  8.0406e-02,
          -1.3566e-01, -1.3011e-01],
         [-8.6594e-06,  8.7894e-07, -5.6089e-06,  ..., -4.8429e-05,
           7.3761e-05,  6.3330e-05],
         [ 1.7285e-04, -1.9222e-04,  3.7253e-05,  ...,  3.5167e-04,
          -1.9968e-04, -2.7008e-05],
         [ 1.4454e-04,  4.5449e-05, -4.2375e-06,  ...,  1.5199e-04,
          -1.8440e-05, -9.9093e-05]]], grad_fn=<SubBackward0>)